In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("D:\\excel data\\covid_toy.csv")


In [3]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
df["city"].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [7]:
df["cough"].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [8]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [9]:
x_train,x_test,y_train,y_test = train_test_split(df.drop("has_covid",axis=1),df["has_covid"],test_size = 0.3,random_state = 1)

In [10]:
x_train.shape

(70, 5)

In [26]:
y_train.shape

(70,)

# Simple Imputing

In [11]:
si = SimpleImputer()

In [12]:
x_train_fever = si.fit_transform(x_train[["fever"]])
x_test_fever = si.fit_transform(x_test[["fever"]])

In [13]:
x_train_fever.shape

(70, 1)

# Ordinal Encoding

In [14]:
from sklearn.preprocessing import OrdinalEncoder

In [15]:
od = OrdinalEncoder(categories=[["Mild","Strong"]])

In [16]:
x_train_cough = od.fit_transform(x_train[["cough"]])
x_test_cough = od.fit_transform(x_test[["cough"]])

In [17]:
x_train_cough.shape

(70, 1)

# OneHot Encoder

In [18]:
ohe = OneHotEncoder(drop="first",sparse=False)

In [19]:
x_train_gender_city = ohe.fit_transform(x_train[["gender","city"]])
x_test_gender_city = ohe.fit_transform(x_test[["gender","city"]])

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
x_train_gender_city.shape

(70, 4)

In [21]:
#Extracting Age
x_train_age = x_train.drop(columns=["gender","fever","cough","city"]).values
x_test_age = x_test.drop(columns=["gender","fever","cough","city"]).values

In [22]:
x_train_age.shape

(70, 1)

In [23]:
x_train_transform = np.concatenate((x_train_age,x_train_cough,x_train_fever,x_train_gender_city),axis=1)
x_test_transform = np.concatenate((x_test_age,x_test_cough,x_test_fever,x_test_gender_city),axis=1)

In [24]:
x_train_transform.shape

(70, 7)

# Column transform

In [27]:
from sklearn.compose import ColumnTransformer

In [39]:
ct = ColumnTransformer(transformers=[
    ("tnsf1",SimpleImputer(),["fever"]),
    ("tnsf2",OrdinalEncoder(categories=[["Mild","Strong"]]),["cough"]),
    ("tnsf3",OneHotEncoder(sparse=False,drop="first"),["gender","city"])
    ],remainder = "passthrough")

In [41]:
ct.fit_transform(x_train).shape

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(70, 7)

In [43]:
ct.transform(x_test).shape

(30, 7)